In [ ]:
Creation of dataframe of metadata for CoughVID

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

folder = '/content/drive/My Drive/Colab Notebooks/coughvid/public_dataset'
if not os.path.exists(folder):
  print(folder + ' does not exist')

In [5]:
import pandas as pd

df_coughvid = pd.read_csv(os.path.join(folder, 'metadata_compiled.csv'))

In [6]:
df_coughvid.head()

,uuid,datetime,cough_detected,SNR,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status,quality_1,cough_type_1,dyspnea_1,wheezing_1,stridor_1,choking_1,congestion_1,nothing_1,diagnosis_1,severity_1,quality_2,cough_type_2,dyspnea_2,wheezing_2,stridor_2,choking_2,congestion_2,nothing_2,diagnosis_2,severity_2,quality_3,cough_type_3,dyspnea_3,wheezing_3,stridor_3,choking_3,congestion_3,nothing_3,diagnosis_3,severity_3,quality_4,cough_type_4,dyspnea_4,wheezing_4,stridor_4,choking_4,congestion_4,nothing_4,diagnosis_4,severity_4
0,00014dcc-0f06-4c27-8c7b-737b18a2cf4c,2020-11-25T18:58:50.488301+00:00,0.0155,7.326171,48.9,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00039425-7f3a-42aa-ac13-834aaa2b6b92,2020-04-13T21:30:59.801831+00:00,0.9609,16.151433,31.3,34.8,15.0,male,False,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0007c6f1-5441-40e6-9aaf-a761d8f2da3b,2020-10-18T15:38:38.205870+00:00,0.1643,16.217201,NaN,NaN,46.0,female,False,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,2020-04-12T04:02:18.159383+00:00,0.9301,20.146058,40.0,-75.1,34.0,male,True,False,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0012c608-33d0-4ef7-bde3-75a0b1a0024e,2020-04-15T01:03:59.029326+00:00,0.0482,0.000000,-16.5,-71.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Remove unwanted columns

In [16]:
df_coughvid = df_coughvid.loc[:,['uuid','cough_detected','age','gender','status']]
print(df_coughvid.shape)
df_coughvid.head()


(27550, 5)


,uuid,cough_detected,age,gender,status
0,00014dcc-0f06-4c27-8c7b-737b18a2cf4c,0.0155,NaN,NaN,NaN
1,00039425-7f3a-42aa-ac13-834aaa2b6b92,0.9609,15.0,male,healthy
2,0007c6f1-5441-40e6-9aaf-a761d8f2da3b,0.1643,46.0,female,healthy
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,0.9301,34.0,male,healthy
4,0012c608-33d0-4ef7-bde3-75a0b1a0024e,0.0482,NaN,NaN,NaN


Create dataframe of audio files. Directory includes json and csv files so this is filtering them out.

In [27]:
fnames = [fname for fname in os.listdir(folder)]

df_fnames = pd.DataFrame(columns =['uuid','audio_file'])

for fname in fnames:
  split = fname.split(".")
  uuid = split[0]
  extension = split[1]
  if extension == 'webm' or extension == 'ogg': 
    df_fnames = df_fnames.append({'uuid':uuid, 'audio_file':fname}, ignore_index=True)
print(df_fnames.shape)
df_fnames.head()

(27550, 2)


,uuid,audio_file
0,fb8cc128-d324-4e2a-bd4d-9eb9ae7b6706,fb8cc128-d324-4e2a-bd4d-9eb9ae7b6706.webm
1,fb8ea05a-e6fc-4083-bfb9-a045fbe89413,fb8ea05a-e6fc-4083-bfb9-a045fbe89413.webm
2,fb8f5c0b-93e0-4403-baea-26673d650b42,fb8f5c0b-93e0-4403-baea-26673d650b42.webm
3,fb911a93-e312-4ae2-9706-e4ac7cb9e491,fb911a93-e312-4ae2-9706-e4ac7cb9e491.webm
4,fb93839b-9bb0-4570-9deb-ee9962d57206,fb93839b-9bb0-4570-9deb-ee9962d57206.webm


Add a column to the coughvid dataframe for the audio file.

In [28]:
left = df_coughvid.set_index(['uuid'])
right = df_fnames.set_index(['uuid'])
df_coughvid = left.join(right)
print(df_coughvid.shape)
df_coughvid.head()

(27550, 5)


,cough_detected,age,gender,status,audio_file
uuid,,,,,
00014dcc-0f06-4c27-8c7b-737b18a2cf4c,0.0155,NaN,NaN,NaN,00014dcc-0f06-4c27-8c7b-737b18a2cf4c.webm
00039425-7f3a-42aa-ac13-834aaa2b6b92,0.9609,15.0,male,healthy,00039425-7f3a-42aa-ac13-834aaa2b6b92.webm
0007c6f1-5441-40e6-9aaf-a761d8f2da3b,0.1643,46.0,female,healthy,0007c6f1-5441-40e6-9aaf-a761d8f2da3b.webm
0009eb28-d8be-4dc1-92bb-907e53bc5c7a,0.9301,34.0,male,healthy,0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm
0012c608-33d0-4ef7-bde3-75a0b1a0024e,0.0482,NaN,NaN,NaN,0012c608-33d0-4ef7-bde3-75a0b1a0024e.webm


Check how many rows there are of each status.

In [29]:
print(df_coughvid.groupby('status').status.count())

status
COVID-19        1155
healthy        12479
symptomatic     2590
Name: status, dtype: int64


Remove the rows where the status is unknown and check I still have the same number of records with a status that is not unknown.

In [30]:
df_coughvid = df_coughvid.dropna(subset=['status'])
print(df_coughvid.groupby('status').status.count())
print(df_coughvid.shape)
df_coughvid.head()

status
COVID-19        1155
healthy        12479
symptomatic     2590
Name: status, dtype: int64
(16224, 5)


,cough_detected,age,gender,status,audio_file
uuid,,,,,
00039425-7f3a-42aa-ac13-834aaa2b6b92,0.9609,15.0,male,healthy,00039425-7f3a-42aa-ac13-834aaa2b6b92.webm
0007c6f1-5441-40e6-9aaf-a761d8f2da3b,0.1643,46.0,female,healthy,0007c6f1-5441-40e6-9aaf-a761d8f2da3b.webm
0009eb28-d8be-4dc1-92bb-907e53bc5c7a,0.9301,34.0,male,healthy,0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm
001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,0.9968,21.0,male,healthy,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f.webm
001d8e33-a4af-4edb-98ba-b03f891d9a6c,0.0306,NaN,female,COVID-19,001d8e33-a4af-4edb-98ba-b03f891d9a6c.webm


The status symptomatic means that they had symptoms but do not know if they have COVID-19 or not. If we do not know the result we cannot include it in the training so remove those rows and check COVID-19 and healthy numbers remain the same.

In [31]:
df_coughvid = df_coughvid[df_coughvid['status'] != 'symptomatic']
print(df_coughvid.groupby('status').status.count())
print(df_coughvid.shape)
df_coughvid.head()

status
COVID-19     1155
healthy     12479
Name: status, dtype: int64
(13634, 5)


,cough_detected,age,gender,status,audio_file
uuid,,,,,
00039425-7f3a-42aa-ac13-834aaa2b6b92,0.9609,15.0,male,healthy,00039425-7f3a-42aa-ac13-834aaa2b6b92.webm
0007c6f1-5441-40e6-9aaf-a761d8f2da3b,0.1643,46.0,female,healthy,0007c6f1-5441-40e6-9aaf-a761d8f2da3b.webm
0009eb28-d8be-4dc1-92bb-907e53bc5c7a,0.9301,34.0,male,healthy,0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm
001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,0.9968,21.0,male,healthy,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f.webm
001d8e33-a4af-4edb-98ba-b03f891d9a6c,0.0306,NaN,female,COVID-19,001d8e33-a4af-4edb-98ba-b03f891d9a6c.webm


The cough_detected column is the probability that the recording contains cough sounds (Orlandic et al, 2021). Hence remove all entries with less than a 50% chance of a cough.

In [32]:
df_coughvid = df_coughvid[df_coughvid['cough_detected'] > 0.5]
print(df_coughvid.groupby('status').status.count())
print(df_coughvid.shape)
df_coughvid.head()

status
COVID-19     782
healthy     9774
Name: status, dtype: int64
(10556, 5)


,cough_detected,age,gender,status,audio_file
uuid,,,,,
00039425-7f3a-42aa-ac13-834aaa2b6b92,0.9609,15.0,male,healthy,00039425-7f3a-42aa-ac13-834aaa2b6b92.webm
0009eb28-d8be-4dc1-92bb-907e53bc5c7a,0.9301,34.0,male,healthy,0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm
001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,0.9968,21.0,male,healthy,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f.webm
001e2f19-d81c-4029-b33c-d2db56b23a4a,0.7811,20.0,male,healthy,001e2f19-d81c-4029-b33c-d2db56b23a4a.webm
0028b68c-aca4-4f4f-bb1d-cb4ed5bbd952,0.8937,28.0,female,healthy,0028b68c-aca4-4f4f-bb1d-cb4ed5bbd952.webm


Save dataframe

In [33]:
folder = '/content/drive/My Drive/Colab Notebooks/coughvid'
df_coughvid.to_csv(os.path.join(folder, 'df_coughvid.csv'))